<div class="alert" style="background-color:#006400; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:15px 15px; color:#FFFFFF; font-size:32px'>HyperTuning for 180 Datasets</h1></div>

The work is under the **"Master Thesis"** by **Chau Tran** with the supervision from **Prof. Roland Olsson**.

## 1. Packages and Datasets

In [177]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, MaxPooling2D,AveragePooling2D,GlobalMaxPooling2D, GlobalAveragePooling2D, Flatten, Dropout, Reshape, BatchNormalization, ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorboard
import kerastuner as kt #(kt.tuners.RandomSearch, kt.tuners.Hyperband)
from kerastuner_tensorboard_logger import (
    TensorBoardLogger,
    setup_tb  # Optional
)
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler

from functools import partial
from matplotlib import rc, style
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
from scipy import stats

import sys, os, math, time, datetime

print("kt: ", kt.__version__)
print("tf: ", tf.__version__)
print(os.getcwd())

%matplotlib inline
%config InlineBackend.figure_format='retina'

style.use("seaborn")
pd.plotting.register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale = 1)

# rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

tf.debugging.set_log_device_placement(False)

strategy = tf.distribute.MirroredStrategy()

# Writing to file
filepath = './Version9.128timesteps/seqnetdata.ni=3.no=5.mc=15.numTimeSteps128.version9.4.csv'
with open(filepath, "r") as fp:
    [noInput, noOutput] = [int(x) for x in fp.readline().split(',')]
rdf = np.array(pd.read_csv(filepath, skiprows=1))
print(type(rdf), rdf.shape)
print(type(noInput), noInput, type(noOutput), noOutput)
display(rdf)

kt:  1.0.2
tf:  2.6.0
/home/ifeai/ChauTran/git/0_HIOF_Studying/0_MasterProject
<class 'numpy.ndarray'> (9999, 1024)
<class 'int'> 3 <class 'int'> 5


array([[0, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [1, 0, 0, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 1, 0]])

## 2. Data Preparation

In [178]:
# np.set_printoptions(threshold=1000)
def shifting(bitlist):
    out = 0
    for bit in bitlist:
        out = (out << 1) | bit
    return out

print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')
df_val, df_train = train_test_split(rdf,test_size=0.5)
print(df_train.shape, df_val.shape)

print('Step 2: Separating values and labels.')
# Training set
x_train, y_train, x_val, y_val = None, None, None, None
for i in range(df_train.shape[0]):
    df_train_i = df_train[i].reshape(-1, noInput+noOutput).astype('float32')
    x_train_i, y_train_i = df_train_i[:, 0:noInput], df_train_i[-1, noInput:]
    x_train = x_train_i[np.newaxis,:,:] if x_train is None else np.append(x_train, x_train_i[np.newaxis,:,:], axis=0)
    y_train = y_train_i.reshape(1, -1) if y_train is None else np.append(y_train, y_train_i.reshape(1, -1), axis=0)
print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)

# Validating set
x_test, y_test = None, None
for i in range(df_val.shape[0]):
    df_val_i = df_val[i].reshape(-1, noInput+noOutput).astype('float32')
    x_val_i, y_val_i = df_val_i[:, 0:noInput], df_val_i[-1, noInput:]
    x_val = x_val_i[np.newaxis,:,:] if x_val is None else np.append(x_val, x_val_i[np.newaxis,:,:], axis=0)
    y_val = y_val_i.reshape(1,-1) if y_val is None else np.append(y_val, y_val_i.reshape(1,-1), axis=0)
print("+ Validating set: ", x_val.shape, y_val.shape, x_val.dtype)

print('Step 3: Normalizing the labels.')
# Training set
y_train_norm = []
for i in range(y_train.shape[0]):
    y_train_norm.append(shifting(y_train[i,:].astype('int32')))
    
enc = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(np.array(y_train_norm).reshape(-1, 1))
y_train_norm = enc.transform(np.array(y_train_norm).reshape(-1, 1))
print("+ Normalizied training set:   ", y_train_norm.shape)
# Validating set
y_val_norm = []
for i in range(y_val.shape[0]):
    y_val_norm.append(shifting(y_val[i,:].astype('int32')))
    
enc = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(np.array(y_val_norm).reshape(-1, 1))
y_val_norm = enc.transform(np.array(y_val_norm).reshape(-1, 1))
print("+ Normalizied validating set: ", y_val_norm.shape)

Step 1: Dividing the training and testing set with ratio 1:1 (50%).
(5000, 1024) (4999, 1024)
Step 2: Separating values and labels.
+ Training set:    (5000, 128, 3) (5000, 5) float32
+ Validating set:  (4999, 128, 3) (4999, 5) float32
Step 3: Normalizing the labels.
+ Normalizied training set:    (5000, 32)
+ Normalizied validating set:  (4999, 32)


## 3. LSTM Model with original outputs (5 bits)

In [179]:
# with strategy.scope(): 
lstm_model = tf.keras.Sequential()
lstm_model.add(
  tf.keras.layers.LSTM(
      units=128, 
      input_shape=[x_train.shape[1], x_train.shape[2]],
      activation='tanh', recurrent_activation='sigmoid',
      unroll =False,
      use_bias=True,
      recurrent_dropout=0,
      return_sequences=False
  )
)

lstm_model.add(tf.keras.layers.Dense(y_train.shape[1], activation='tanh'))
adam = tf.optimizers.Adam(learning_rate = 0.001, decay=1e-6)
# lstm_model.compile(loss='mse', optimizer=adam, metrics='mse')
lstm_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.51)])

strt_time = datetime.datetime.now() 
lstm_model.fit(
    x_train, y_train,
    epochs=8,
    batch_size=1,
    verbose=0,
    validation_data=(x_val, y_val),
    shuffle=True,
    use_multiprocessing=True
)
curr_time = datetime.datetime.now()
display(lstm_model.summary())
timedelta = curr_time - strt_time
dnn_train_time = timedelta.total_seconds()

val_performance = lstm_model.evaluate(x_val, y_val)

Model: "sequential_54"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_54 (LSTM)               (None, 128)               67584     
_________________________________________________________________
dense_57 (Dense)             (None, 5)                 645       
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


None

157/157 [==============================] - 1s 6ms/step - loss: 0.2828 - binary_accuracy: 0.8030


In [180]:
y_pred = lstm_model.predict(x_val, verbose=0)
print(y_pred)

m = tf.keras.metrics.BinaryAccuracy()
m.update_state(y_pred, y_val)
print(m.result().numpy())

y_pred[np.where(y_pred < 0)] = 0
y_pred[np.where(y_pred >= 0)] = 1
display(y_pred.astype('int32'))
display(y_val.astype('int32'))
count = 0
for i in range(y_pred.shape[0]):
    if (np.array_equal(y_pred[i].astype('int32'), y_val[i].astype('int32'))):
        count += 1
print(count, count/y_pred.shape[0], val_performance[1], val_performance[1]*y_pred.shape[0])

[[-0.19788855  0.6051079  -0.5205557   0.99930775  0.5333794 ]
 [ 0.99878234  0.50712156 -0.7296089   0.99404347  0.5857842 ]
 [ 0.99945194  0.44372046  0.99650085  0.99919295  0.55372477]
 ...
 [ 0.9923825   0.48660094  0.99015915  0.99815124  0.5038921 ]
 [-0.68550926  0.585049    0.9999517  -0.9407191   0.38284588]
 [ 0.9945059   0.48318762  0.98883027  0.99833316  0.50074506]]
0.0


array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       ...,
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]], dtype=int32)

array([[0, 1, 0, 1, 1],
       [1, 1, 0, 1, 0],
       [1, 1, 1, 1, 1],
       ...,
       [1, 1, 1, 1, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 1, 1, 0]], dtype=int32)

175 0.035007001400280055 0.8030405640602112 4014.3997797369957


## 3. LSTM Model with normalized outputs (32 bits)

In [159]:
# with strategy.scope(): 
lstm_model = tf.keras.Sequential()
lstm_model.add(
  tf.keras.layers.LSTM(
      units=8, 
      input_shape=[x_train.shape[1], x_train.shape[2]],
      activation='tanh', recurrent_activation='sigmoid',
      return_sequences=False #many2many
  )
)
lstm_model.add(tf.keras.layers.Dense(y_train_norm.shape[1], activation='tanh'))
adam = tf.optimizers.Adam(learning_rate = 0.001, decay=1e-6)
lstm_model.compile(loss='mse', optimizer=adam, metrics=[tf.keras.metrics.CategoricalAccuracy()])

strt_time = datetime.datetime.now() 
lstm_model.fit(
    x_train, y_train_norm,
    epochs=50,
    batch_size=128,
    verbose=0,
    validation_data=(x_val, y_val_norm),
    shuffle=True,
    use_multiprocessing=True
)
curr_time = datetime.datetime.now()
display(lstm_model.summary())
timedelta = curr_time - strt_time
dnn_train_time = timedelta.total_seconds()

val_performance = lstm_model.evaluate(x_val, y_val_norm)

Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_49 (LSTM)               (None, 8)                 384       
_________________________________________________________________
dense_52 (Dense)             (None, 32)                288       
Total params: 672
Trainable params: 672
Non-trainable params: 0
_________________________________________________________________


None

157/157 [==============================] - 0s 3ms/step - loss: 0.0298 - categorical_accuracy: 0.0548


In [160]:
y_pred = lstm_model.predict(x_val, verbose=0)
m = tf.keras.metrics.CategoricalAccuracy()
m.update_state(y_pred, y_val_norm)
print(m.result().numpy(), m.result().numpy()*y_pred.shape[0])
display(y_pred)
print(m.variables)
y_pred[np.where(y_pred <= 0.0505)] = 0
y_pred[np.where(y_pred > 0.0505)] = 1
display(y_pred.astype('int32'))
display(y_val_norm.astype('int32'))
count = 0
for i in range(y_pred.shape[0]):
    if (np.array_equal(y_pred[i].astype('int32'), y_val_norm[i].astype('int32'))):
        count += 1
print(count, count/y_pred.shape[0], val_performance)

0.054810964 274.00000689178705


array([[ 3.9581209e-05, -4.1223639e-03,  1.0322663e-03, ...,
         6.5011114e-02,  6.1142743e-02,  5.7384141e-02],
       [ 2.9469490e-02,  1.2287834e-02,  2.7728166e-02, ...,
         4.1147072e-02,  4.6307910e-02,  3.7659619e-02],
       [ 2.4722844e-02,  2.3733180e-02,  2.1713689e-02, ...,
         4.9571559e-02,  4.8341978e-02,  2.8172469e-02],
       ...,
       [ 2.3851139e-02,  1.4238188e-02,  2.3083949e-02, ...,
         3.9092626e-02,  5.2784853e-02,  3.0109696e-02],
       [ 2.2479740e-03, -8.3094891e-03,  4.3173889e-03, ...,
         6.3332014e-02,  5.9263784e-02,  6.2414411e-02],
       [-4.6912581e-05, -8.2772933e-03,  3.1036984e-03, ...,
         6.7349046e-02,  5.7406075e-02,  6.8269171e-02]], dtype=float32)

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=274.0>, <tf.Variable 'count:0' shape=() dtype=float32, numpy=4999.0>]


array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]], dtype=int32)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

9 0.0018003600720144029 [0.0298045352101326, 0.054810963571071625]
